# 0. Install required library

In [ ]:
%pip install ipykernal
%pip install librosa
%pip install numpy
%pip install noisereduce
%pip install sklearn
%pip install pickle

In [1]:
import pickle
import librosa
import numpy as np
import noisereduce as nr
import sklearn.preprocessing

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from tqdm import tqdm

c:\Users\GAILAB\anaconda3\envs\fmcc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Preprocess data

In [2]:
# Make avg_frame length for same input
def zero_padding(mfcc, avg_frame=40):
    if mfcc.shape[1] < avg_frame:
        padding = np.zeros((mfcc.shape[0], avg_frame - mfcc.shape[1]))
        mfcc = np.hstack((mfcc, padding))
    else:
        mfcc = mfcc[:, :avg_frame]
        
    return mfcc

# raw-to-mfcc
def raw_to_mfcc(raw_file_path):

    # Read raw file
    with open(raw_file_path, 'rb') as file:
        # Raw-to-wav
        audio_data = np.frombuffer(file.read(), dtype=np.int16)
        audio_data = audio_data.astype(np.float32)

    # Reduce noise
    clean_data = nr.reduce_noise(y=audio_data, sr=16000)

    # Wav-to-mfcc
    mfcc = librosa.feature.mfcc(y=clean_data, sr=16000, n_mfcc=20, n_mels=100, fmin=0, fmax=400)

    # Zero padding and scaling
    mfcc = zero_padding(mfcc, avg_frame=37)
    mfcc = sklearn.preprocessing.minmax_scale(mfcc, axis=1)

    # Flatten data for model input
    mfcc = mfcc.flatten()

    return mfcc

In [3]:
X_train = []
Y_train = []

# Read the training data and labels from the control file name
with open('fmcc_train.ctl', 'r') as file:
    for line in tqdm(file):
        file_path = './raw16k/train/' + line.rstrip() + '.raw'

        # Append label
        Y_train.append(line[0])

        # Raw-to-mfcc
        mfcc = raw_to_mfcc(file_path)
        X_train.append(mfcc)

Y_train = np.array(Y_train)

82it [00:03, 27.92it/s]

# 2. Train Model & Save model

In [ ]:
# Create SVM, KNN and Random Forest classifiers
svm_classifier = SVC()
knn_classifier = KNeighborsClassifier(n_neighbors=13)
rf = RandomForestClassifier(
    min_samples_leaf=5,
    min_samples_split=7,
    max_depth=17,
    criterion='gini',
    max_features='log2',
    bootstrap=False,
    random_state=42
)

# Create a voting classifier
voting_classifier = VotingClassifier(
    estimators=[('svm', svm_classifier),
                ('knn', knn_classifier),
                ('rf', rf)],
    voting='hard'
)

# Train the voting classifier
voting_classifier.fit(X_train, Y_train)

# Save the voting classifier
with open('voting_classifier.pkl', 'wb') as f:
    pickle.dump(voting_classifier, f)